In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/BRAIN_TUMOR_TASK/Brain_Tumor_Detection/TRAIN',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/BRAIN_TUMOR_TASK/Brain_Tumor_Detection/TRAIN',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/BRAIN_TUMOR_TASK/Brain_Tumor_Detection/TEST',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2412 images belonging to 2 classes.
Found 602 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
76/76 [==============================] - 350s 5s/step - loss: 0.5232 - accuracy: 0.8006 - val_loss: 0.2283 - val_accuracy: 0.9186
Epoch 2/5
76/76 [==============================] - 271s 4s/step - loss: 0.2258 - accuracy: 0.9175 - val_loss: 0.1272 - val_accuracy: 0.9485
Epoch 3/5
76/76 [==============================] - 268s 4s/step - loss: 0.1009 - accuracy: 0.9681 - val_loss: 0.0693 - val_accuracy: 0.9767
Epoch 4/5
76/76 [==============================] - 264s 3s/step - loss: 0.0539 - accuracy: 0.9855 - val_loss: 0.0623 - val_accuracy: 0.9817
Epoch 5/5
76/76 [==============================] - 267s 4s/step - loss: 0.0382 - accuracy: 0.9871 - val_loss: 0.0379 - val_accuracy: 0.9917


In [ ]:
model.save("Model.h5","label.txt")

In [ ]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [ ]:
image = Image.open("/content/drive/MyDrive/Brain_Tumor_Detection/test/pred/pred37.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

In [ ]:
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)